In [ ]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import requests
from pypdf import PdfReader
import io
import os
import tempfile

In [ ]:
def read_doc(pdf_url):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()
        
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_file_path = tmp_file.name
        loader = PyPDFLoader(tmp_file_path)
        documents = loader.load()
        
        return documents
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the PDF: {e}")
    except Exception as e:
        print(f"Error processing the PDF: {e}")

In [ ]:
def chunk_data(docs, chunk_size = 800, chunk_overlap = 50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [ ]:
link = "https://www.indiabudget.gov.in/doc/budget_speech.pdf"
doc = read_doc(link)
doc
#len(doc)

In [ ]:
os.environ['OPENAI_API_KEY'] = 'sk-proj-7JM7dbkrCqKOWux9s1S2T3BlbkFJQKo8xZXsvRLSbfVGNiCF'
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
vectors = embeddings.embed_query("how are you?")
len(vectors)

In [ ]:
os.environ['PINECONE_API_KEY'] = 'fdea4408-8b2b-4f96-9f7f-83c9e355a2aa'
from pinecone import Pinecone
Pinecone(pi_key=os.environ.get("PINECONE_API_KEY"))


In [ ]:
from langchain.vectorstores import Pinecone
index = Pinecone.from_documents(doc, embeddings, index_name = "business-qa-bot")

In [ ]:
def retrieve_query(query, k = 2):
    matching_results = index.similarity_search(query, k = k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm = OpenAI(model_name = "text-embedding-ada-002", temperature = 0.5)
chain = load_qa_chain(llm, chain_type = "stuff")

In [ ]:
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question = query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer  = retrieve_answers(our_query)
print(answer)

In [ ]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import requests
from pypdf import PdfReader
import io
import os
import tempfile
def read_doc(pdf_url):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()
        
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_file_path = tmp_file.name
        loader = PyPDFLoader(tmp_file_path)
        documents = loader.load()
        
        return documents
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the PDF: {e}")
    except Exception as e:
        print(f"Error processing the PDF: {e}")


def chunk_data(docs, chunk_size = 800, chunk_overlap = 50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc
link = "https://www.indiabudget.gov.in/doc/budget_speech.pdf"
doc = read_doc(link)
doc
#len(doc)

os.environ['OPENAI_API_KEY'] = 'sk-proj-7JM7dbkrCqKOWux9s1S2T3BlbkFJQKo8xZXsvRLSbfVGNiCF'
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

vectors = embeddings.embed_query("how are you?")
len(vectors)

os.environ['PINECONE_API_KEY'] = 'fdea4408-8b2b-4f96-9f7f-83c9e355a2aa'
from pinecone import Pinecone
Pinecone(pi_key=os.environ.get("PINECONE_API_KEY"))

from langchain.vectorstores import Pinecone
index = Pinecone.from_documents(doc, embeddings, index_name = "business-qa-bot")

def retrieve_query(query, k = 2):
    matching_results = index.similarity_search(query, k = k)
    return matching_results

from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

llm = OpenAI(model_name = "text-embedding-ada-002", temperature = 0.5)
chain = load_qa_chain(llm, chain_type = "stuff")

def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question = query)
    return response

our_query = "How much the agriculture target will be increased by how many crore?"
answer  = retrieve_answers(our_query)
print(answer)